In [1]:
import pandas as pd
import numpy as np 

In [2]:
url = "./breast-cancer.csv"
data = pd.read_csv(url)
data['diagnosis'] = data['diagnosis'].replace({'M': 1, 'B': 0})
# Lấy cột cần di chuyển
column_to_move = data['diagnosis']
data.drop(columns=["id"], inplace=True)
# Loại bỏ cột này khỏi DataFrame
data.drop(columns=['diagnosis'], inplace=True)
# Thêm cột này lại vào cuối DataFrame
data['diagnosis'] = column_to_move
data.head(20)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_33592\348588340.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['diagnosis'] = data['diagnosis'].replace({'M': 1, 'B': 0})


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.26540,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.18600,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.24300,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.25750,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678,1
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.17410,0.3985,0.12440,1
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368,1
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510,1
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.20600,0.4378,0.10720,1
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.22100,0.4366,0.20750,1


In [3]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index # Chỉ số đặc trưng dùng để phân chia
        self.threshold = threshold # Ngưỡng giá trị để phân chia
        self.left = left # Cây con bên trái
        self.right = right   # Cây con bên phải
        self.info_gain = info_gain # Thông tin đạt được khi phân chia
        self.value = value  # Giá trị của nút nếu là lá (leaf node)

In [4]:
class DecisionTreeID3:
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None  # Gốc của cây
        self.min_samples_split = min_samples_split  # Số lượng mẫu tối thiểu để phân chia nút
        self.max_depth = max_depth  # Độ sâu tối đa của cây
    
    # Phương thức xây dựng cây
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # Kiểm tra điều kiện dừng
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            # Nếu có phân chia tốt, tiếp tục xây dựng cây con
            if best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # Nếu không có phân chia tốt, trả về giá trị lá
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    # Phương thức tính toán thông tin đạt được
    def information_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        return gain
    
    # Phương thức tính toán entropy
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    # Phương thức tìm phân chia tốt nhất
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
          
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                   
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    # Phương thức phân chia tập dữ liệu
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right
    
    # Phương thức tính toán giá trị lá
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)
    
    # Phương thức in cây
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)
        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    # Phương thức huấn luyện cây
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    # Phương thức dự đoán cho tập dữ liệu mới
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    # Phương thức dự đoán cho một điểm dữ liệu
    def make_prediction(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [5]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [6]:
classifier = DecisionTreeID3()
classifier.fit(X_train, Y_train)
classifier.print_tree()

X_7 <= 0.04908 ? 0.5516364837002203
 left:X_20 <= 16.77 ? 0.13232456637189233
  left:X_13 <= 48.29 ? 0.04640094813817412
    left:0.0
    right:1.0
  right:X_1 <= 19.46 ? 0.5548894409556178
    left:0.0
    right:1.0
 right:X_23 <= 861.5 ? 0.2712979618346121
  left:X_21 <= 25.47 ? 0.5409813230754511
    left:0.0
    right:1.0
  right:1.0


In [8]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Độ chính xác của mô hình trên tập kiểm tra: {accuracy * 100:.2f}%")
from sklearn.metrics import recall_score
recall = recall_score(Y_test, Y_pred)
print(f"Độ nhạy của mô hình trên tập kiểm tra: {recall * 100:.2f}%")
from sklearn.metrics import precision_score
precision = precision_score(Y_test, Y_pred)
print(f"Độ chính xác dự đoán của mô hình trên tập kiểm tra: {precision * 100:.2f}%")
from sklearn.metrics import f1_score
f1 = f1_score(Y_test, Y_pred)
print(f"F1 Score của mô hình trên tập kiểm tra: {f1 * 100:.2f}%")

Độ chính xác của mô hình trên tập kiểm tra: 95.61%
Độ nhạy của mô hình trên tập kiểm tra: 97.50%
Độ chính xác dự đoán của mô hình trên tập kiểm tra: 90.70%
F1 Score của mô hình trên tập kiểm tra: 93.98%


min_samples_leaf:Số lượng mẫu tối thiểu có mặt trong một lá (leaf node)

max_depth: Độ sâu tối đa của cây.

min_samples_split: Số lượng mẫu tối thiểu cần có để một nút có thể được chia tách

max_leaf_nodes: Số lượng lá tối đa mà cây có thể có

min_impurity_decrease: Độ giảm tối thiểu của độ bất thuần (impurity) cần thiết để một nút được phân chia